In [113]:
import dicom
import numpy as np
from stl import mesh
import plotly.offline as py

import plotly.graph_objs as go
py.offline.init_notebook_mode(connected=True)

In [123]:
study='patient4'
plan = dicom.read_file("/media/sf_CRCT/JupyterNotebook/data/RTplanReader/rp/rp_"+study+"_e9.dcm")
print(plan)
#print len(plan.BeamSequence[0].BlockSequence[0].BlockData)
bdata= [float(val) for val in plan.BeamSequence[0].BlockSequence[0].BlockData]
x=[]
y=[]

for i in np.arange(0, len(bdata)):
    if i %2  ==0:
        x.append(bdata[i])
    else: 
        y.append(bdata[i])
trace1 = go.Scatter(
    x = x,
    y = y,
    mode = 'markers', 
    name = 'Block edge', 
    )
data1 = [trace1]
fig1= go.Figure(data=data1)   
py.iplot(fig1)
#print plan.BeamSequence[0].BlockSequence[0].BlockData

(0008, 0005) Specific Character Set              CS: 'ISO_IR 192'
(0008, 0012) Instance Creation Date              DA: '20181023'
(0008, 0013) Instance Creation Time              TM: '181830.844000'
(0008, 0016) SOP Class UID                       UI: RT Plan Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.246.352.71.5.387430965438.1049851.20181023181554
(0008, 0020) Study Date                          DA: '20161219'
(0008, 0030) Study Time                          TM: '093209'
(0008, 0050) Accession Number                    SH: '103055776'
(0008, 0060) Modality                            CS: 'RTPLAN'
(0008, 0070) Manufacturer                        LO: 'Varian Medical Systems'
(0008, 0090) Referring Physician's Name          PN: 'MODESTO^ANOUCHKA'
(0008, 1010) Station Name                        SH: 'safran5'
(0008, 1030) Study Description                   LO: 'TDM therapie SPC'
(0008, 103e) Series Description                  LO: 'Plans ARIA RadOnc'
(0008, 1070) Op

In [124]:
pts_up=[]

for i in range(len(x)):
    pt=[]
    pt.append(x[i])
    pt.append(y[i])
    pt.append(7.5)
    pts_up.append(pt)

print pts_up[0]
origin1=[0, 0, 7.5]

stl_top=[]
for i in range(0, len(pts_up)):
    f=[]
    z=i+1
    
    if z > len(pts_up)-1: 
        f.append(pts_up[0])
    else:
        f.append(pts_up[z])
    f.append(pts_up[i])
    f.append(origin1)
    stl_top.append(f)
print stl_top[0]
print stl_top[-1]

[-75.0, 75.0, 8.5]
[[75.0, 75.0, 8.5], [-75.0, 75.0, 8.5], [0, 0, 7.5]]
[[-75.0, 75.0, 8.5], [-75.0, -75.0, 8.5], [0, 0, 7.5]]


In [125]:
pts_down=[]
for i in range(len(x)):
    pt=[]
    pt.append(x[i])
    pt.append(y[i])
    pt.append(-7.5)
    pts_down.append(pt)
      
origin2=[0, 0, -7.5] 

stl_bot=[]
for i in range(0, len(pts_down)):
    f=[]
    z=i+1
    f.append(pts_down[i])
    if z > len(pts_down)-1: 
        f.append(pts_down[0])
    else:
        f.append(pts_down[z])
    f.append(origin2)
    stl_bot.append(f)
print stl_bot[0]
print stl_bot[-1]

[[-75.0, 75.0, -8.5], [75.0, 75.0, -8.5], [0, 0, -7.5]]
[[-75.0, -75.0, -8.5], [-75.0, 75.0, -8.5], [0, 0, -7.5]]


In [126]:
vertices = np.asarray([stl_top])
vertices.resize((len(stl_top),3,3))

vertices1 = np.asarray([stl_bot])
vertices1.resize((len(stl_bot),3,3))

#insert_down = mesh.Mesh(np.zeros(vertices1.shape[0], dtype=mesh.Mesh.dtype))
# for i, f in enumerate(vertices1):
#     insert_down.vectors[i]=f
# insert_down.save('insert_down3.stl')
# insert_up = mesh.Mesh(np.zeros(vertices.shape[0], dtype=mesh.Mesh.dtype))
# for i, f in enumerate(vertices):
#     insert_up.vectors[i]=f
# insert_up.save('insert_up3.stl')

In [127]:
pts_side=[]
stl_side=[]

for i in range(len(stl_top)):
    fa=[]
    fb=[]
    fa.append(stl_top[i][0])    
    if i == len(stl_top)-1:
            fa.append(stl_bot[0][0])
    else:
        fa.append(stl_bot[i+1][0])
    fa.append(stl_bot[i][0])
    
    fb.append(stl_top[i][0])
    if i == len(stl_top)-1:
        fb.append(stl_top[0][0])
        fb.append(stl_bot[0][0])
                
    else:
        fb.append(stl_top[i+1][0])
        fb.append(stl_bot[i+1][0])
        
    stl_side.append(fa)
    stl_side.append(fb)     
vertice2=np.asarray(stl_side)


# insert_side = mesh.Mesh(np.zeros(vertice2.shape[0], dtype=mesh.Mesh.dtype))
# for i, f in enumerate(vertice2):
#      insert_side.vectors[i]=f
# insert_side.save('insert_side3.stl')

In [128]:
tot_len=vertices.shape[0]+vertices1.shape[0]+vertice2.shape[0]
vertex=np.zeros(tot_len)
vertex.resize((tot_len,3 ,3))

for i, f in enumerate(vertices): 
    vertex[i]=f
for i in range(len(vertices), len(vertices)+len(vertices1)):
    vertex[i]=vertices1[i-len(vertices)]
for i in range((len(vertices)+len(vertices1)),tot_len):
    vertex[i]=vertice2[i-(len(vertices)+len(vertices1))]

insert = mesh.Mesh(np.zeros(tot_len, dtype=mesh.Mesh.dtype))
for i, f in enumerate(vertex):
    insert.vectors[i]=f
#insert.save('/media/sf_CRCT/JupyterNotebook/data/RTplanReader/stl/'+study+'_insert.stl')
insert.save('/media/sf_Downloads/stl/'+study+'_e9_insert.stl')